In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_boston

import pickle
import json

import warnings
warnings.filterwarnings("ignore")

## Problem Statement

## Data Gathering 

In [2]:
df = pd.read_csv("medical_insurance.csv")
df

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## Exploratory Data Analysys

In [3]:
df.shape

(1338, 7)

#### 1. Missing Values 

In [4]:
df.isna().sum()    

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


### 1.Sex

In [6]:
df["sex"].unique()

array(['female', 'male'], dtype=object)

In [7]:
df["sex"].nunique()

2

In [8]:
df["sex"].value_counts()

male      676
female    662
Name: sex, dtype: int64

In [9]:
df["sex"].replace({"male":0,"female":1},inplace=True)
sex_dict = {"male":0,"female":1}
sex_dict

{'male': 0, 'female': 1}

### smoker

In [10]:
df["smoker"].value_counts()

no     1064
yes     274
Name: smoker, dtype: int64

In [11]:
df["smoker"].replace({"no":0,"yes":1},inplace=True)
smoker_dict = {"no":0,"yes":1}
smoker_dict

{'no': 0, 'yes': 1}

### region

In [12]:
df["region"].value_counts()
df

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,southwest,16884.92400
1,18,0,33.770,1,0,southeast,1725.55230
2,28,0,33.000,3,0,southeast,4449.46200
3,33,0,22.705,0,0,northwest,21984.47061
4,32,0,28.880,0,0,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,northwest,10600.54830
1334,18,1,31.920,0,0,northeast,2205.98080
1335,18,1,36.850,0,0,southeast,1629.83350
1336,21,1,25.800,0,0,southwest,2007.94500


In [13]:
df = pd.get_dummies(df,columns=["region"])
df

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,1,27.900,0,1,16884.92400,0,0,0,1
1,18,0,33.770,1,0,1725.55230,0,0,1,0
2,28,0,33.000,3,0,4449.46200,0,0,1,0
3,33,0,22.705,0,0,21984.47061,0,1,0,0
4,32,0,28.880,0,0,3866.85520,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...
1333,50,0,30.970,3,0,10600.54830,0,1,0,0
1334,18,1,31.920,0,0,2205.98080,1,0,0,0
1335,18,1,36.850,0,0,1629.83350,0,0,1,0
1336,21,1,25.800,0,0,2007.94500,0,0,0,1


In [14]:
df.isna().mean()*100

age                 0.0
sex                 0.0
bmi                 0.0
children            0.0
smoker              0.0
charges             0.0
region_northeast    0.0
region_northwest    0.0
region_southeast    0.0
region_southwest    0.0
dtype: float64

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               1338 non-null   int64  
 1   sex               1338 non-null   int64  
 2   bmi               1338 non-null   float64
 3   children          1338 non-null   int64  
 4   smoker            1338 non-null   int64  
 5   charges           1338 non-null   float64
 6   region_northeast  1338 non-null   uint8  
 7   region_northwest  1338 non-null   uint8  
 8   region_southeast  1338 non-null   uint8  
 9   region_southwest  1338 non-null   uint8  
dtypes: float64(2), int64(4), uint8(4)
memory usage: 68.1 KB


In [16]:
df.describe()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
count,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,0.494768,30.663397,1.094918,0.204783,13270.422265,0.242152,0.242900,0.272048,0.242900
std,14.049960,0.500160,6.098187,1.205493,0.403694,12110.011237,0.428546,0.428995,0.445181,0.428995
min,18.000000,0.000000,15.960000,0.000000,0.000000,1121.873900,0.000000,0.000000,0.000000,0.000000
25%,27.000000,0.000000,26.296250,0.000000,0.000000,4740.287150,0.000000,0.000000,0.000000,0.000000
50%,39.000000,0.000000,30.400000,1.000000,0.000000,9382.033000,0.000000,0.000000,0.000000,0.000000
75%,51.000000,1.000000,34.693750,2.000000,0.000000,16639.912515,0.000000,0.000000,1.000000,0.000000
max,64.000000,1.000000,53.130000,5.000000,1.000000,63770.428010,1.000000,1.000000,1.000000,1.000000


#### Split train and test data 

In [17]:
x = df.drop("charges",axis=1)
y = df["charges"]

x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3,random_state=10)
x_train.shape

(936, 9)

In [18]:
knn_reg = KNeighborsRegressor(n_neighbors=10)  # n_neighbour = 5, P= 2
knn_reg.fit(x_train, y_train)

KNeighborsRegressor(n_neighbors=10)

### Model Evaluation

In [20]:
# Testing Data Evaluation
y_pred = knn_reg.predict(x_test)

mse = mean_squared_error(y_test, y_pred)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_test, y_pred)
print("MAE :",mae)

r2_value = r2_score(y_test, y_pred)
print("R-Score :",r2_value)

MSE : 115109993.50737438
RMSE : 10728.93254277304
MAE : 7971.191337557214
R-Score : 0.11356945151447351


In [21]:
# Training Data Evaluation
y_pred_train = knn_reg.predict(x_train)

mse = mean_squared_error(y_train, y_pred_train)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_train, y_pred_train)
print("MAE :",mae)

r2_value = r2_score(y_train, y_pred_train)
print("R-Score :",r2_value)

MSE : 105942329.28910479
RMSE : 10292.829022630503
MAE : 7655.723471164531
R-Score : 0.3102868084021211


# Normalization

In [23]:
normal_scalar = MinMaxScaler()

normal_scalar.fit(x_train)  # Store min and max values 

MinMaxScaler()

In [24]:
# To get training accuracy

array = normal_scalar.transform(x_train)  # Actual application the formalae
array

x_train_normal = pd.DataFrame(array, columns=x_train.columns)

# Model Training
knn_reg = KNeighborsRegressor(n_neighbors=7, p=1)  # n_neighbour = 5, P= 2
knn_reg.fit(x_train_normal, y_train)

y_pred_train = knn_reg.predict(x_train_normal)

mse = mean_squared_error(y_train, y_pred_train)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_train, y_pred_train)
print("MAE :",mae)

r2_value = r2_score(y_train, y_pred_train)
print("R-Score :",r2_value)

MSE : 24625803.3858348
RMSE : 4962.439257646868
MAE : 3085.6692168681325
R-Score : 0.839679365529556


In [25]:
# To get testing accuracy

array = normal_scalar.transform(x_test)

x_test_normal = pd.DataFrame(array, columns=x_test.columns)

y_pred = knn_reg.predict(x_test_normal)

mse = mean_squared_error(y_test, y_pred)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_test, y_pred)
print("MAE :",mae)

r2_value = r2_score(y_test, y_pred)
print("R-Score :",r2_value)

MSE : 33580917.95246842
RMSE : 5794.904481738109
MAE : 3450.8149613230275
R-Score : 0.7414025436692684


### 2. Standardization

In [26]:
std_scalar = StandardScaler()

std_scalar.fit(x_train)  # Min , std are stored

StandardScaler()

In [27]:
# getting accuracy on training data

array = std_scalar.transform(x_train)  # Formulae applied

x_train_std = pd.DataFrame(array, columns=x_train.columns)

# Model Training
knn_reg_best = KNeighborsRegressor(n_neighbors=7, p=1)  # n_neighbour = 5, P= 2
knn_reg_best.fit(x_train_std, y_train)

y_pred_train = knn_reg_best.predict(x_train_std)

mse = mean_squared_error(y_train, y_pred_train)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_train, y_pred_train)
print("MAE :",mae)

r2_value = r2_score(y_train, y_pred_train)
print("R-Score :",r2_value)

MSE : 23075437.981438756
RMSE : 4803.690038026887
MAE : 2966.1668481196584
R-Score : 0.8497726632546883


In [28]:
# getting accuracy on testing data

array = std_scalar.transform(x_test)

x_test_std = pd.DataFrame(array, columns=x_test.columns)

y_pred = knn_reg_best.predict(x_test_std)

mse = mean_squared_error(y_test, y_pred)
print("MSE :", mse)

rmse = np.sqrt(mse)
print("RMSE :",rmse)

mae = mean_absolute_error(y_test, y_pred)
print("MAE :",mae)

r2_value = r2_score(y_test, y_pred)
print("R-Score :",r2_value)

MSE : 29498487.136964787
RMSE : 5431.2509734834375
MAE : 3200.80625496553
R-Score : 0.7728402258085629


In [36]:
age = 67
sex = "male"
bmi = 27.9
children = 3
smoker = "yes"
region = "southeast"

# charges = ?

In [46]:
column_names=x.columns

In [47]:
print(sex_dict)
print(smoker_dict)
print(column_names)

{'male': 0, 'female': 1}
{'no': 0, 'yes': 1}
Index(['age', 'sex', 'bmi', 'children', 'smoker', 'region_northeast',
       'region_northwest', 'region_southeast', 'region_southwest'],
      dtype='object')


In [49]:
project_data = {"sex":sex_dict, "smoker":smoker_dict, "columns":list(column_names)}
project_data

{'sex': {'male': 0, 'female': 1},
 'smoker': {'no': 0, 'yes': 1},
 'columns': ['age',
  'sex',
  'bmi',
  'children',
  'smoker',
  'region_northeast',
  'region_northwest',
  'region_southeast',
  'region_southwest']}

In [50]:
project_data['smoker'][smoker]

1

In [51]:
# Column name
region = "region_" + region
region

'region_southeast'

In [52]:
region_index = np.where(column_names == region)[0][0]
region_index

7

In [53]:
array = np.zeros(x.shape[1])
array[0] = age
array[1] = project_data['sex'][sex]
array[2] = bmi
array[3] = children
array[4] = project_data['smoker'][smoker]
array[region_index] = 1

print(array)

[67.   0.  27.9  3.   1.   0.   0.   1.   0. ]


In [54]:
predicted_insurance = knn_reg.predict([array])[0]
print("Pridicted medical insurance is ",predicted_insurance)

Pridicted medical insurance is  47175.97364285715


In [55]:
import pickle

with open("KNN_medical_Model.pkl", "wb") as f:
    pickle.dump(knn_reg, f)

In [56]:
with open("Scalar.pkl", "wb") as f:
    pickle.dump(std_scalar, f)

In [57]:
import json
with open("project_data.json", "w") as f:
    json.dump(project_data, f)

In [58]:
json_data = {'sex': {'female': 0, 'male': 1},
 'smoker': {'yes': 0, 'no': 1},
 'columns': ['age',
  'sex',
  'bmi',
  'children',
  'smoker',
  'region_northeast',
  'region_northwest',
  'region_southeast',
  'region_southwest']}
len(json_data['columns'])

9